In [ ]:
import numpy as np
from scipy.io import wavfile
from utils import load_surf, load_config, butterworth_highpass, zscore, get_envelope, lanczosinterp2D, get_logger, get_embeddings, downsample_embeddings_lanczos
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import nibabel as nib
from nilearn import datasets, surface, plotting
import os
import time

cfg = load_config()
DATADIR = cfg['DATA_DIR']
STIMDIR = cfg['STIM_DIR']
FIGDIR = cfg['FIG_DIR']

logging = get_logger()

subject = 'sub-007'
session = 'ses-02'
task = 'christmas1940'
hemi = 'L'

tr = 1.5

fsaverage = datasets.fetch_surf_fsaverage('fsaverage6')

surf_file = os.path.join(DATADIR, subject, session, 'func', f'{subject}_{session}_task-{task}_hemi-{hemi}_space-fsaverage6_bold.func.gii')
if not os.path.exists(surf_file):
    logging.error(f"Surface file {surf_file} does not exist.")

surf_data = surface.load_surf_data(surf_file)
sdata = np.array(surf_data, dtype=np.float64)

sdata = zscore(butterworth_highpass(sdata, tr, 0.01), axis=1)

# load wav
wav_file = os.path.join(STIMDIR, f'{task}.wav')
sr, audio = wavfile.read(wav_file)

if audio.ndim > 1:
    audio_envelope = get_envelope(audio)[:, 0]
else:
    audio_envelope = get_envelope(audio)

audio_envelope_avg = np.mean(audio_envelope[:len(audio_envelope) // int(sr * tr) * int(sr * tr)].reshape(-1, int(sr * tr)), axis=1)

zEnv = zscore(audio_envelope_avg)

sdata = sdata[:, 2:zEnv.shape[0]+2]

# load embeddings
embs, starts, stops = get_embeddings(task)
n_features = embs.shape[-1]
emdata = downsample_embeddings_lanczos(embs, starts, stops, sdata.shape[-1], tr)

# trim first and last 10 trs; zscore
xdata = zscore(emdata[10:-10], axis=0)
ydata = zscore(sdata[:, 10:-10], axis=1)

/var/folders/r2/rgb930tj5ss_cmb9qkb2mfym0000gn/T/ipykernel_82215/3317762594.py:38: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sr, audio = wavfile.read(wav_file)
